In [29]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
import pickle

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 32)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [10]:
model = Net()
model.load_state_dict(torch.load('mnist_dnn.pt'));

In [11]:
print("> printing the model")
print(model)

> printing the model
Net(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)


In [13]:
print("> showing how many trainable parameters there are in the model")
print( sum(p.numel() for p in model.parameters() if p.requires_grad) )

> showing how many trainable parameters there are in the model
25450


#### Model summary
```
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x) <== skip for inference
        x = self.fc2(x)
```

## Manually dividing the model into contracts
- **Future: build graph parser for pytorch models and auto-generate deployable cairo contracts + deployment script for address linkage, taking StarkNet constraints + parallelization scheme intended by the user**
- FC1: X(1x784) * M1 (784x32) = V1 (1x32)
    - dividing fc1 into 32 sop (sum-of-product; each being 784 multiply-add + bias), each producing one element in V1
    - further dividing each sop into 8 sub-sop, first 7 having 100 terms each, while the 8th has 84 terms.
        - **if not dividing into sub-sop's => would trigger exception on Lark's recursion depth limit; Lark is the parser used by the Cairo compiler by StarkWare**
    - one contract per sub-sop
    - **mnist_v1_sop<0-31>.cairo** one contract per sop, grouping 8 sub-sop's
    - **mnist_v1.cairo**: one contract groups 32 sop's to V1
    - => 1 + 32 + 32\*8 = **289 contracts**
- RELU: V1 (1x32) ==relu==> H1 (1x32)
    - one contract to produce H1
    - => **1 contract**
- FC2: H1 (1x32) * M2 (32x10) = Z (1x10)
    - dividing fc2 into 10 sop, each producing one element in Z
    - one contract per sop (32 multiply-add + bias)
    - => **10 contracts**
- Total: **300 contracts**

### Contract: V1

In [14]:
print("> Printing out cairo code for top-level contracts")

# for i in range(32):
#     print(f'    let (addr_{i}) = stored_addresses.read({i})')
#     print(f'    local pedersen_ptr : HashBuiltin* = pedersen_ptr')
#     print(f'    let (v1_{i}) = IContractV1SOP{i}.compute(addr_{i}, 784, x)')
#     print(f'    assert [v1+{i}] = v1_{i}')
#     print()

# for i in range(32):
#     print(f'@contract_interface')
#     print(f'namespace IContractV1SOP{i}:')
#     print(f'    func compute(x_len : felt, x : felt*) -> (res : felt):')
#     print(f'    end')
#     print(f'end')
#     print()

> Printing out cairo code for top-level contracts


### Contract: V1_SOP<0-31>

In [15]:
part1 = [
'%lang starknet',
'%builtins pedersen range_check',
'',
'from starkware.cairo.common.cairo_builtins import HashBuiltin',
'from starkware.cairo.common.math import signed_div_rem',
'',
'@storage_var',
'func stored_addresses (idx : felt) -> (addr : felt):',
'end',
'',
'@external',
'func admin_store_addresses {',
'        syscall_ptr : felt*, pedersen_ptr : HashBuiltin*, range_check_ptr',
'    } (idx : felt, addr : felt) -> ():',
'    stored_addresses.write(idx, addr)',
'    return ()',
'end',
'',
'################################################',
''
]

############

def part2 (sop_idx):
    ret = []
    for i in range(8):
        ret += [
            '@contract_interface',
            f'namespace IContractV1SOP{sop_idx}SUB{i}:',
            '    func compute(x_len : felt, x : felt*) -> (res : felt):',
            '    end',
            'end',
            ''
        ]
    ret.append('################################################')
    ret.append('')
    return ret

############

def part3 (sop_idx, quantize_depth=8):
    ret = []
    ret += [
        '@view',
        'func compute {',
        '        syscall_ptr : felt*, pedersen_ptr : HashBuiltin*, range_check_ptr',
        '    }(',
        '        x_len : felt,',
        '        x : felt*',
        '    ) -> (',
        '        res : felt',
        '    ):',
        '    alloc_locals',
        ''
    ]
    
    for i in range(8):
        x_len = 100 if i!=7 else 84
        ret.append(f'    let (addr_{i}) = stored_addresses.read({i})')
        ret.append(f'    local pedersen_ptr : HashBuiltin* = pedersen_ptr')
        ret.append(f'    let (local sub{i}) = IContractV1SOP{sop_idx}SUB{i}.compute(addr_{i}, {x_len}, x+100*{i})')
        ret.append('')
    ret.append('    let res_ = sub0 + sub1 + sub2 + sub3 + sub4 + sub5 + sub6 + sub7')
    ret.append(f'    let (res, _) = signed_div_rem(res_, {10**quantize_depth}, 2 ** 64)')
    ret.append('    return (res)')
    ret.append('end')
    ret.append('')

    return ret

def gen_v1_sop_contract(sop_idx, quantize_depth):
    ret = []
    ret += part1
    ret += part2(sop_idx)
    ret += part3(sop_idx, quantize_depth)
    
    with open(f'gen_contract/mnist_v1_sop{sop_idx}.cairo', 'a') as f:
        for line in ret:
            f.write(line+'\n')

In [23]:
print("> Generating 32 sop contracts for V1")
# for sop_idx in range(32):
#     gen_v1_sop_contract(sop_idx, quantize_depth=8)

> Generating 32 sop contracts for V1


### Contract: V1_SOP<0-31>_SUBSOP<0-7>
- subsop0 - subsop6: adding 100 product terms
- subsop7: adding 84 product terms + bias

#### Contract structure
```
%lang starknet
%builtins pedersen range_check

from starkware.cairo.common.cairo_builtins import HashBuiltin
from starkware.cairo.common.alloc import alloc

@view
func compute {
        syscall_ptr : felt*, pedersen_ptr : HashBuiltin*, range_check_ptr
    }(
        x_len : felt,
        x : felt*
    ) -> (
        res : felt
    ):
    
    let res = [x+0]*___ + [x+1]*___ + .... + [x+99]*___
    
    return (res)
end
```

In [274]:
def gen_v1_sop_subsop_contract(sop_idx, subsop_idx, model, quantize_depth = 8):
    b_s = model.fc1.bias.tolist()
    b = b_s[sop_idx]
    w_list = model.fc1.weight[sop_idx].tolist()
    f_quantize = lambda e,n : int(e* 10**n)
    w_list_quantized = [f_quantize(w, quantize_depth) for w in w_list]
    b_quantized = quantize(b, quantize_depth)
    ret = []
    
    ret += [
        '%lang starknet',
        '%builtins pedersen range_check',
        '',
        'from starkware.cairo.common.cairo_builtins import HashBuiltin',
        ''
    ]
    
    ret += [
        '@view',
        'func compute {',
        '        syscall_ptr : felt*, pedersen_ptr : HashBuiltin*, range_check_ptr',
        '    }(',
        '        x_len : felt,',
        '        x : felt*',
        '    ) -> (',
        '        res : felt',
        '    ):',
        ''
    ]
    if subsop_idx == 7:
        ## add 84 product terms and bias
        w_list_quantized_ = w_list_quantized[700 : 783]
        product_terms = [f'[x+{i}] * {w}' for i,w in enumerate(w_list_quantized_)]
        expression = ' + '.join([str(b_quantized)] + product_terms)
    else:
        ## add 100 product terms
        w_list_quantized_ = w_list_quantized[subsop_idx*100 : (subsop_idx+1)*100-1]
        product_terms = [f'[x+{i}] * {w}' for i,w in enumerate(w_list_quantized_)]
        expression = ' + '.join(product_terms)
    ret.append(f'    let res = {expression}')
    
    ret += [
        '    return (res)',
        'end'
        ''
    ]
    
    with open(f'gen_contract/mnist_v1_sop{sop_idx}_subsop{subsop_idx}.cairo', 'a') as f:
        for line in ret:
            f.write(line+'\n')
            
    return ret

In [22]:
print("> Generating 8 subsop contracts for each of the 32 sop's")
# for sop_idx in range(32):
#     for subsop_idx in range(8):
#         ret = gen_v1_sop_subsop_contract(sop_idx, subsop_idx, model, quantize_depth = 8)

> Generating 8 subsop contracts for each of the 32 sop's


### relu contract

In [21]:
print("> Generating cairo code for relu contract")
# for i in range(32):
#     print(f'    let (h) = _relu([v1+{i}])')
#     print(f'    assert [h1+{i}] = h')
#     print()

> Generating cairo code for relu contract


### Z contract

In [25]:
print("> Generating cairo code for the Z (FC2) contract")
# for i in range(10):
#     print(f'    let (addr_{i}) = stored_addresses.read({i})')
#     print(f'    local pedersen_ptr : HashBuiltin* = pedersen_ptr')
#     print(f'    let (z_{i}) = IContractZSOP{i}.compute(addr_{i}, 32, h1)')
#     print(f'    assert [z+{i}] = z_{i}')
#     print()

> Generating cairo code for the z (FC2) contract


### z_sop<0-9> contracts

In [275]:
def gen_z_sop_contract(sop_idx, model, quantize_depth = 8):
    b_s = model.fc2.bias.tolist()
    b = b_s[sop_idx]
    w_list = model.fc2.weight[sop_idx].tolist()
    f_quantize = lambda e,n : int(e* 10**n)
    w_list_quantized = [f_quantize(w, quantize_depth) for w in w_list]
    b_quantized = quantize(b, quantize_depth)
    ret = []
    
    ret += [
        '%lang starknet',
        '%builtins pedersen range_check',
        '',
        'from starkware.cairo.common.cairo_builtins import HashBuiltin',
        ''
    ]
    
    ret += [
        '@view',
        'func compute {',
        '        syscall_ptr : felt*, pedersen_ptr : HashBuiltin*, range_check_ptr',
        '    }(',
        '        x_len : felt,',
        '        x : felt*',
        '    ) -> (',
        '        res : felt',
        '    ):',
        ''
    ]
    
    product_terms = [f'[x+{i}] * {w}' for i,w in enumerate(w_list_quantized)]
    expression = ' + '.join([str(b_quantized)] + product_terms)
    ret.append(f'    let res = {expression}')
    
    ret += [
        '    return (res)',
        'end'
        ''
    ]
    
    with open(f'gen_contract/mnist_z_sop{sop_idx}.cairo', 'a') as f:
        for line in ret:
            f.write(line+'\n')
            
    return ret

In [26]:
print("> Generating 10 sop contracts for Z contract")
# for i in range(10):
#     gen_z_sop_contract(i, model, 8);

> Generating 10 sop contracts for Z contract


-------

### Training

In [237]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args['dry_run']:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def run():
    args = {
        'no_cuda' : True,
        'seed' : 1,
        'batch_size' : 64,
        'test_batch_size' : 1000,
        'lr' : 1.0,
        'gamma' : 0.7,
        'epochs' : 14,
        'save_model' : True,
        'log_interval' : 10,
        'dry_run' : False
        
    }
   
    use_cuda = not args['no_cuda'] and torch.cuda.is_available()

    torch.manual_seed(args['seed'])

    device = torch.device("cuda" if use_cuda else "cpu")

    train_kwargs = {'batch_size': args['batch_size']}
    test_kwargs = {'batch_size': args['test_batch_size']}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args['lr'])

    scheduler = StepLR(optimizer, step_size=1, gamma=args['gamma'])
    for epoch in range(1, args['epochs'] + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    if args['save_model']:
        torch.save(model.state_dict(), "mnist_dnn.pt")

In [58]:
run()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.329856
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.573720
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.413218
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.955432
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.786503
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.881180
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.771456
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.804304
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.143172
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.575278
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.671376
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.671880
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.625420
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.604873
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.773807
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.766047
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.657894
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.557043
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.810708
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.812729
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.461188
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.600052
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.580971
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.632555
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.320015
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.392818
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.312475
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.463928
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.524607
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.414306
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.191782
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.566535
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.429980
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.474902
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.421944
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.590989
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.486295
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.444422
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.918812
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.260515


Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.387406
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.387632
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.627937
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.442815
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.556867
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.204338
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.364329
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.449830
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.330669
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.625873
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.464646
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.338473
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.240435
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.320967
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.527491
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.282453
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.605719
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.473271
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.482670
Train Epoch: 4 [42880/60000 (71%)]	Loss: 0.489779


Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.270076
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.802154
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.507357
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.173361
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.285443
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.587554
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.459232
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.277767
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.325778
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.385050
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.282043
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.277236
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.600683
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.505492
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.234278
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.356704
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.398274
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.572113
Train Epoch: 6 [26880/60000 (45%)]	Loss: 0.566408
Train Epoch: 6 [27520/60000 (46%)]	Loss: 0.472434



Test set: Average loss: 0.2007, Accuracy: 9426/10000 (94%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.328883
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.481213
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.532987
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.208293
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.415460
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.406048
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.194627
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.262633
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.625460
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.477707
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.376385
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.176200
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.236408
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.473929
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.286555
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.437849
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.481544
Train Epoch: 8 [10880/60000 (18%)]	Loss: 0.350946
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.934134
Train Epoch: 8 [12

Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.358094
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.517971
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.415541
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.276986
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.364222
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.231730
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.334715
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.383083
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.219380
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.254993
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.187745
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.310694
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.294213
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.323858
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.235604
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.561703
Train Epoch: 9 [56320/60000 (94%)]	Loss: 0.321078
Train Epoch: 9 [56960/60000 (95%)]	Loss: 0.294328
Train Epoch: 9 [57600/60000 (96%)]	Loss: 0.488895
Train Epoch: 9 [58240/60000 (97%)]	Loss: 0.197956


Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.476216
Train Epoch: 11 [28800/60000 (48%)]	Loss: 0.339454
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.273320
Train Epoch: 11 [30080/60000 (50%)]	Loss: 0.601405
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.410095
Train Epoch: 11 [31360/60000 (52%)]	Loss: 0.578066
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.673122
Train Epoch: 11 [32640/60000 (54%)]	Loss: 0.537931
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.490873
Train Epoch: 11 [33920/60000 (57%)]	Loss: 0.246622
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.420692
Train Epoch: 11 [35200/60000 (59%)]	Loss: 0.400236
Train Epoch: 11 [35840/60000 (60%)]	Loss: 0.310979
Train Epoch: 11 [36480/60000 (61%)]	Loss: 0.327893
Train Epoch: 11 [37120/60000 (62%)]	Loss: 0.396460
Train Epoch: 11 [37760/60000 (63%)]	Loss: 0.455136
Train Epoch: 11 [38400/60000 (64%)]	Loss: 0.220084
Train Epoch: 11 [39040/60000 (65%)]	Loss: 0.356075
Train Epoch: 11 [39680/60000 (66%)]	Loss: 0.585818
Train Epoch: 11 [40320/60000 (6

Train Epoch: 13 [10240/60000 (17%)]	Loss: 0.684186
Train Epoch: 13 [10880/60000 (18%)]	Loss: 0.326011
Train Epoch: 13 [11520/60000 (19%)]	Loss: 0.866696
Train Epoch: 13 [12160/60000 (20%)]	Loss: 0.481567
Train Epoch: 13 [12800/60000 (21%)]	Loss: 0.307755
Train Epoch: 13 [13440/60000 (22%)]	Loss: 0.239095
Train Epoch: 13 [14080/60000 (23%)]	Loss: 0.552454
Train Epoch: 13 [14720/60000 (25%)]	Loss: 0.648483
Train Epoch: 13 [15360/60000 (26%)]	Loss: 0.295497
Train Epoch: 13 [16000/60000 (27%)]	Loss: 0.591951
Train Epoch: 13 [16640/60000 (28%)]	Loss: 0.344244
Train Epoch: 13 [17280/60000 (29%)]	Loss: 0.233474
Train Epoch: 13 [17920/60000 (30%)]	Loss: 0.299240
Train Epoch: 13 [18560/60000 (31%)]	Loss: 0.553337
Train Epoch: 13 [19200/60000 (32%)]	Loss: 0.473008
Train Epoch: 13 [19840/60000 (33%)]	Loss: 0.282226
Train Epoch: 13 [20480/60000 (34%)]	Loss: 0.263909
Train Epoch: 13 [21120/60000 (35%)]	Loss: 0.302602
Train Epoch: 13 [21760/60000 (36%)]	Loss: 0.150758
Train Epoch: 13 [22400/60000 (3

Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.238084
Train Epoch: 14 [54400/60000 (91%)]	Loss: 0.382641
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.320666
Train Epoch: 14 [55680/60000 (93%)]	Loss: 0.610193
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.375267
Train Epoch: 14 [56960/60000 (95%)]	Loss: 0.226622
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.633872
Train Epoch: 14 [58240/60000 (97%)]	Loss: 0.241344
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.149917
Train Epoch: 14 [59520/60000 (99%)]	Loss: 0.158832

Test set: Average loss: 0.1959, Accuracy: 9436/10000 (94%)



---

### Datastructure exploration

In [72]:
b_s = model.fc1.bias.tolist()
b_0 = b_s[0]
w_0_list = model.fc1.weight[0].tolist()

In [73]:
b_0

-0.03493674099445343

In [74]:
w_0_list

[0.02740933559834957,
 -0.006756555289030075,
 0.0020833066664636135,
 0.02577025257050991,
 -0.024615662172436714,
 0.03042559139430523,
 0.001659609959460795,
 0.027176501229405403,
 0.01397180836647749,
 0.0046339379623532295,
 0.018912842497229576,
 0.010768760927021503,
 0.01650378294289112,
 -0.011312826536595821,
 0.006403053179383278,
 0.005791680887341499,
 0.01418349239975214,
 0.008864249102771282,
 0.04022698849439621,
 0.02012101747095585,
 -0.004293275065720081,
 -0.012563132680952549,
 0.003020653733983636,
 -0.006398078054189682,
 -0.0024376793298870325,
 0.010716964490711689,
 0.03029737062752247,
 0.028419116511940956,
 -0.025905447080731392,
 0.03114704042673111,
 0.01898438297212124,
 0.0428837388753891,
 0.032580479979515076,
 -0.02377472072839737,
 -0.04583946242928505,
 -0.03219585120677948,
 0.019051160663366318,
 -0.01690705679357052,
 0.0018371023470535874,
 -0.02316276542842388,
 0.012664335779845715,
 -0.04223133996129036,
 0.021850770339369774,
 -0.01896392

In [32]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
ds = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
IDX = 5
d = ds.data[IDX].tolist()
d_flatten = []
for i in range(28):
    for j in range(28):
        print("%3d" % d[i][j], end=" ")
        d_flatten.append(d[i][j])
    print()

# print( len(d_flatten) )
# print('[%s]' % ','.join( [str(e) for e in d_flatten] ))

  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  13  25 100 122   7   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0  33 151 208 252 252 252 146   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0  40 152 244 252 253 224 211 252 232  40   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0  15 152 239 252 252 252 216  31  37 252 252  60   0   0   0 

/Users/guiltygyoza/opt/anaconda3/envs/py37/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [34]:
print("> Investigating FC1 output collected from StarkNet contract for precision loss")
sol_s = [28255435953, -16423638096, -463597710, 13498191087, -30114092289, 45508329950, 26430724623, -7281002187, -28972881867, -37352675430, -22022185386, -33144123396, -16708183373, -1922013630, 25706864873, -3648043006, 22349402213, 20949610819, -25246367889, 19770606824, -3150400393, 86447757560, -32367798672, -14193289090, 72816301052, -42810611972, -1820129923, 13129317624, -7273541488, 97032159880, -54870506948, 49598410122]

for sol in sol_s:
    print(sol / 10**8)

> Investigating FC1 output collected from StarkNet contract for precision loss
282.55435953
-164.23638096
-4.6359771
134.98191087
-301.14092289
455.0832995
264.30724623
-72.81002187
-289.72881867
-373.5267543
-220.22185386
-331.44123396
-167.08183373
-19.2201363
257.06864873
-36.48043006
223.49402213
209.49610819
-252.46367889
197.70606824
-31.50400393
864.4775756
-323.67798672
-141.9328909
728.16301052
-428.10611972
-18.20129923
131.29317624
-72.73541488
970.3215988
-548.70506948
495.98410122


In [291]:
idx_s = range(10)
img_array_s = []
for idx in idx_s:
    d = ds.data[idx].tolist()
    d_flatten = []
    for i in range(28):
        for j in range(28):
            d_flatten.append(d[i][j])
    img_array_s.append(d_flatten)

with open("img_array_s.txt", "wb") as fp:   #Pickling
    pickle.dump(img_array_s, fp)

In [38]:
sol_s = [-16926425655610295354, -25769741692563623387, -12526590447583294615, -745904275336316246, -28866927377392675394, -1563107310419300343, -19919289842112617675, -21459652795700226659, -9713915730007115903, -13271941145501024253]

print("> Investigating fixed-point precision on starknet-NN output precision")
print("> Scale floats by 10^16:")
for sol in sol_s:
    print(sol / 10**16)


> Investigating fixed-point precision on starknet-NN output precision
> Scale floats by 10^16:
-1692.6425655610296
-2576.9741692563625
-1252.6590447583294
-74.59042753363163
-2886.6927377392676
-156.31073104193004
-1991.9289842112619
-2145.9652795700226
-971.3915730007116
-1327.1941145501023


In [37]:
sol_s = [-169122213668, -257449510423, -125158504463, -7408666570, -288326465022, -15572522167, -198993928326, -214469740798, -96905331675, -132517456612]

print("> Scale floats by 10^8:")
for sol in sol_s:
    print(sol / 10**8)


> Scale floats by 10^8:
-1691.22213668
-2574.49510423
-1251.58504463
-74.0866657
-2883.26465022
-155.72522167
-1989.93928326
-2144.69740798
-969.05331675
-1325.17456612
